In [ ]:
!pip install rudalle==1.0.0
!wget https://www.kaggleusercontent.com/kf/86840742/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..ZvbQWMd2ooIAxEAPDJ3vFg.GS1SW-4V3nAjJOlvs6XhYH0DKtTS433GaSeophK84LMuN7BKAHgnSr-reUOWDmr-7IUcjOCWMhmTlUb_lfMutFWkLx79OvE0QxpEravZALuRRhs-px8irtXSWiP-CmiZ5AsqSiGQmFQofCa2xoo4uQijL3V74KT3IfYDhZ6YsP9yqCFMRMSyIJ3VdhZ_SIQW85PgYqQ6BzwCTOc01AovblrNd2c6xQGf5_WdypWwZXUE92-ZU6N3RoXZNdX6AX8n0Qll7aRgc5JEYLvNF3FRUwC7gl2qdLu1XxD2kGPRbLYfdSo-zsmtB2l2aaJ4GsC3F-qt6Fvp7wrnFy1Kgz-YOlfOYcsGqDWaYmSpIG2o3kVHrLg_Ic5Y9lyWgYTdORxUAeYBR80QdqZ0_rpPJrURm1lAl3JgXzssZUFiMnnYu3Tq4ef-skuR-BxFA1ZB11zlkPHsGR3IlUh2lEwvxsCzsPS2M0sPvTrqN0cm2P56jhaPOqUwGfLosntqwMwUq_JddDZKrpfDqC5TDnwrFD_LskxxgI8WRI3ShqxovqXj8wtJARuAinJcZBWHDuK7maASug7t7iAv2MC5iReM2_9_ScGF9RYzrppblVvgBB0F4QwHXUIZnU_EZ7cMh5FIpBLrTAaT734MFgUQqyoHxHVOOsC2xG7G59pER9RnB5PSMcw.tKZqBy_k4Dh3ShxKObwmpA/model.pth

In [ ]:
from rudalle import get_vae
import torch.nn as nn
from torch.optim import AdamW
import torch
from IPython.display import clear_output

device = "cuda" if torch.cuda.is_available() else "cpu"

vae = get_vae().to(device)
vae.train()
vae.load_state_dict(torch.load("model.pth"))
optimizer = AdamW(vae.parameters(), lr=3e-4)
loss_function = nn.MSELoss().to(device)

torch.cuda.empty_cache()
clear_output()

In [ ]:
from PIL import Image
import numpy as np
import glob
import matplotlib.pyplot as plt

def center_crop(img):
    width, height = img.size
    
    new_width, new_height = (width, width) if width < height else (height, height)
    
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    return img.crop((left, top, right, bottom))

In [ ]:
paths = [path for path in glob.glob("../input/coco-2017-dataset/coco2017/train2017/*")]
print(len(paths))

In [ ]:
import random
import torch

batch_size = 8

total_loss = 0.0

print("starting.....2")


for i in range(0, len(paths) // batch_size + 1):
    images = [center_crop(Image.open(path).convert("RGB")).resize((256, 256), Image.BICUBIC) for path in paths[(i-1) * batch_size:i * batch_size]]

    if len(images) == 0:
        continue

    images = np.array([np.array(img) for img in images])

    torch.cuda.empty_cache()

    images = torch.tensor(images).to(device).float() / 255

    torch.cuda.empty_cache()

    images = torch.permute(images, (0, 3, 1, 2))

    torch.cuda.empty_cache()

    optimizer.zero_grad()

    image_masked = images.detach().clone()

    for im in image_masked:
        for n in range(random.randrange(10, 100)):
            w, h = (random.randrange(10, 50), random.randrange(10, 50))

            box = torch.zeros(3, w, h)

            x = random.randrange(0, 256 - w)
            y = random.randrange(0, 256 - h)

            im[:, x:x+w, y:y+h] = box


    y = vae.decode(vae.get_codebook_indices(image_masked))
    torch.cuda.empty_cache()

    loss = loss_function(y, images)

    clear_output()

    print(f"Steps: {i}\nLoss: {loss.item()}")

    total_loss += loss.item()

    loss.backward()
    optimizer.step()

    torch.cuda.empty_cache()
    
    if i % 1000 == 0:
        torch.save(vae.state_dict(), "model.pth")

torch.save(vae.state_dict(), "model.pth")

total_loss /= (10_000 / batch_size)

print(f"Epoch: {1}\nTotal Loss: {total_loss}")

In [ ]:
#torch.save(vae.state_dict(), "model.pth")

In [ ]:
# !pip install gdown
# !gdown --id 1VxYLMzPaxbn4pk0MlZI9irBNWj8Py2cu

In [ ]:
!wget https://openai.com/content/images/2021/08/openai-avatar.png -O test.jpg

In [ ]:
import random

image = Image.open("./test.jpg").convert("RGB").resize((700,700), Image.BICUBIC)
display(image)

image = np.array(image)

image = (torch.tensor(image).to(device).float() / 255)[None,:]
image = torch.permute(image, (0, 3, 1, 2))

for im in image:
    for n in range(random.randrange(1, 2)):
        w, h = (random.randrange(200, 500), random.randrange(200, 500))

        box = torch.zeros(3, w, h)

        x = random.randrange(0, 700 - w)
        y = random.randrange(0, 700 - h)

        im[:, x:x+w, y:y+h] = box

m = torch.squeeze(image.detach().clone()).permute(1,2,0).detach().cpu().numpy()
m = Image.fromarray((m * 255).astype("uint8"))

display(m)

code = vae.get_codebook_indices(image)
y = vae.decode(code)

m = torch.squeeze(y).permute(1,2,0).detach().cpu().numpy()
m = Image.fromarray((m * 255).astype("uint8"))

display(m)